<img src=https://static.wixstatic.com/media/f6bd9a_6a972d9b31324653bf198c38ce94339e~mv2.png>

# Carrusel de RDBMS
## Ejercicios de SQL resueltos

En este cuaderno resuelvo los ejercicios del 21 al 30 del documento: 'Desarrollo de Aplicaciones en Entornos de Cuarta Generación y Herramientas CASE: Ejercicios de SQL.'

Para ello utilizo una version de la base de datos -que nombre "PRACTICAS_HOSPITAL" modificada para ser compatible en esta ocasion, para **MS** **SQL** **SERVER**, siguiendo la estructura del libro.


Tanto el diagrama de **Entidad-Relacion**, como el **Script** para cargar la base de datos (que permite, se pueda replicar el ejercicio) se encuentran disponibles en el mismo repositorio.

#### Celda para configurar el entorno

In [ ]:
# !pip install pyodbc
# !pip install pandas
# !pip install sqlalchemy

#### Importamos librerias y se realiza la conexion utilizando un archivo json con la configuracion local de la base de datos

Si se desea replicar lo propio usando el script y este cuaderno, basta con crear el archivo json con el par de valores que contengan su informacion local y cargarlo al directorio, o en su defecto construir la cadena directamente.

server:

database:

password:

user:


In [1]:
import json
from sqlalchemy import create_engine
import pandas as pd

# Cargar los detalles de la conexión desde un archivo de configuración
with open('config1.json', 'r') as f:
    config = json.load(f)

# Extraer los datos de conexión del objeto JSON
server = config['server']
database = config['database']
username = config['username']
password = config['password']

# Crear la cadena de conexión
connection_string = f'mssql+pyodbc://{username}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server'

# Crear la conexión
engine = create_engine(connection_string)

#### Realizamos una prueba y se Inspecciona la base de datos "practicas_hospital"

In [2]:
# Definir consulta en SQL compatible con MS SQL Server
try:
    QUERY = """
    SELECT name 
    FROM sys.tables;
    """
# Ejecutar la consulta y obtener los resultados en un DataFrame
    df = pd.read_sql_query(QUERY, engine)
# Mostrar los primeros resultados
    print("La conexión se ha realizado con éxito.\n")
    print(df)
except Exception as e:
    print("Hubo un error al realizar la conexión:", e)

La conexión se ha realizado con éxito.

           name
0      HOSPITAL
1          SALA
2     PLANTILLA
3     OCUPACION
4        DOCTOR
5       ENFERMO
6      EMPLEADO
7  DEPARTAMENTO
8   sysdiagrams


#### Inspeccion a las tablas de la base de datos "practica_hospital"
#### Con la finalidad de observar su estructura y tipo de datos

In [3]:
# se excluye el nombre de la tabla de usuario que alude al diagrama (disponible en el repositorio)
indice = df[df['name'] == 'sysdiagrams'].index
df = df.drop(indice)
for valor in df['name']:
    QUERY = "SELECT COLUMN_NAME,DATA_TYPE, IS_NULLABLE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '" + f'{valor}'+"'"
    df1 = pd.read_sql_query(QUERY, engine)
    print(f'El contenido de la tabla {valor}\n')
    print(df1)
    print()

El contenido de la tabla HOSPITAL

    COLUMN_NAME DATA_TYPE IS_NULLABLE
0  HOSPITAL_COD       int          NO
1        NOMBRE   varchar          NO
2     DIRECCION   varchar          NO
3      TELEFONO   varchar          NO
4      NUM_CAMA       int          NO

El contenido de la tabla SALA

    COLUMN_NAME DATA_TYPE IS_NULLABLE
0       SALA_ID       int          NO
1  HOSPITAL_COD       int          NO
2      SALA_COD       int          NO
3        NOMBRE   varchar         YES
4      NUM_CAMA       int         YES

El contenido de la tabla PLANTILLA

    COLUMN_NAME DATA_TYPE IS_NULLABLE
0  HOSPITAL_COD       int          NO
1      SALA_COD       int          NO
2   EMPLEADO_NO       int          NO
3      APELLIDO   varchar         YES
4       FUNCION   varchar         YES
5         TURNO   varchar         YES
6       SALARIO       int         YES

El contenido de la tabla OCUPACION

    COLUMN_NAME DATA_TYPE IS_NULLABLE
0   INSCRIPCION       int          NO
1  HOSPITAL_COD       i

### Resolucion de los ejercicios numerados del 21 al 30

##### 21. Buscar que departamentos tienen mas de dos personas trabajando en la misma profesión.

In [4]:
QUERY = """
SELECT DEPT_NO, COUNT(*) AS NUM_TRABAJDORES
FROM EMPLEADO
GROUP BY DEPT_NO, OFICIO
HAVING COUNT(*)> 2;
"""
df = pd.read_sql_query(QUERY, engine)
df

,DEPT_NO,NUM_TRABAJDORES
0,30,4


##### 22. Se desea saber el numero de empleados por departamento que tienen por oficio  de "EMPLEADO". La salida debe estar ordenada por el numero de departamento/PNDEDOR', 'EMPLEADO');


In [5]:
QUERY = """
SELECT DEPT_NO, COUNT(*) AS NUM_EMPLEADO
FROM EMPLEADO
WHERE UPPER(OFICIO) = 'EMPLEADO'
GROUP BY DEPT_NO
ORDER BY DEPT_NO;
"""
df = pd.read_sql_query(QUERY, engine)
df

,DEPT_NO,NUM_EMPLEADO
0,10,1
1,20,2
2,30,1


##### 23. Se desea saber el salario total (salario mas comisión) medio anual de los vendedores de nuestra empresa.

In [6]:
QUERY = """
SELECT OFICIO, AVG(SALARIO + ISNULL(COMISION,0)) AS SALARIO_TOTAL
FROM EMPLEADO
GROUP BY OFICIO
HAVING UPPER(OFICIO) = 'VENDEDOR';
"""
df = pd.read_sql_query(QUERY, engine)
df

,OFICIO,SALARIO_TOTAL
0,Vendedor,253500


##### 24. Se desea saber el salario total (salario mas comisión) medio anual, tanto de los empleados como de los vendedores de nuestra empresa.

In [7]:
QUERY = """
SELECT OFICIO, AVG(SALARIO + ISNULL(COMISION, 0)) AS SALARIO_MEDIO
FROM EMPLEADO
GROUP BY OFICIO
HAVING UPPER(OFICIO) IN ('VENDEDOR', 'EMPLEADO');
"""
df = pd.read_sql_query(QUERY, engine)
df

,OFICIO,SALARIO_MEDIO
0,Empleado,134875
1,Vendedor,253500


##### 25. Se desea saber para cada departamento y en cada oficio, el maximo salario y la suma total de salarios, pero solo de aquellos 
departamentos y oficios cuya suma salarial supere o sea igual que el 50% de su maximo salario. En el muestreo, solo se estudiaron 
aquellos empleados que no tienen comisión o la tengan menor que el 25% de su salari DEPT_NO;

In [8]:
QUERY = """
SELECT DEPT_NO, OFICIO, MAX(SALARIO) AS MAXIMO, SUM(SALARIO) AS SUMA
FROM EMPLEADO
WHERE ISNULL(COMISION, 0) < (SALARIO * 0.25)
GROUP BY DEPT_NO, OFICIO
HAVING SUM(SALARIO) >= (MAX(SALARIO) * 0.5)
ORDER BY DEPT_NO;
"""
df = pd.read_sql_query(QUERY, engine)
df

,DEPT_NO,OFICIO,MAXIMO,SUMA
0,10,Director,318500,318500
1,10,Empleado,169000,169000
2,10,Presidente,650000,650000
3,20,Analista,390000,780000
4,20,Director,386750,386750
5,20,Empleado,143000,247000
6,30,Director,370500,370500
7,30,Empleado,123500,123500
8,30,Vendedor,208000,403000


##### 26. Se desea saber para cada oficio, dentro de cada año de alta distinto que existe en nuestra empresa, el numero de empleados y la media salarial que tiene. Para este estudio, no se tendrá en cuenta a los empleados que no hayan sido dados de alta en un día laboral. Además, solo se desea saber estos datos, de aquellos oficios y años que tienen mas de un empleado. La salida debe estar ordenada por el año de alta y la media salarial descendéntemente.



In [9]:
QUERY = """
SELECT DATEPART(YEAR,FECHA_ALTA) AS ANIO, OFICIO, COUNT(*) AS TOT_EMPL, AVG(SALARIO) AS SAL_PROM
FROM EMPLEADO
WHERE DATEPART(weekday, FECHA_ALTA) NOT IN (1, 7)
GROUP BY DATEPART(YEAR, FECHA_ALTA), OFICIO
HAVING COUNT(*) > 1
ORDER BY ANIO, SAL_PROM DESC;
"""
df = pd.read_sql_query(QUERY, engine)
df

,ANIO,OFICIO,TOT_EMPL,SAL_PROM
0,1981,Director,3,358583
1,1981,Vendedor,3,188500


##### 27. Se desea saber, para cada inicial de apellido que exista en la empresa (tratando solo las iniciales consonantes),el maximo salario que tiene asociada. No se tendrá en cuenta en el estudio a aquellos empleados que contengan su apellido mas de una "N". La salida debe estar ordenada por la inicial.


In [10]:
QUERY = """
SELECT SUBSTRING(APELLIDO,1,1) AS INICIAL, MAX (SALARIO) AS MAXIMO
FROM EMPLEADO
WHERE NOT (LEN(APELLIDO) - LEN(REPLACE(UPPER(APELLIDO), 'N', ''))) > 1
GROUP BY SUBSTRING(APELLIDO,1,1)
HAVING NOT SUBSTRING(APELLIDO,1,1) IN ('A','E','I', 'O','U')
ORDER BY INICIAL;
"""
df = pd.read_sql_query(QUERY, engine)
df

,INICIAL,MAXIMO
0,C,318500
1,G,390000
2,J,386750
3,M,169000
4,N,370500
5,R,650000
6,S,162500
7,T,195000


##### 28. Se desea obtener un informe matriz como el que se presenta, en el que la coordenada vertical hace referencia a los distintos oficios existentes en la empresa, y la coordenada horizontal a los distintos departamentos. Los valores de la matriz, indicaran la suma de salarios por oficio y departamento. La ultima columna indica la suma total de salarios por oficio.

In [11]:
# UN EJEMPLO SIMPLE DE MODELOS DE CUBO OLAP (PIVOT)
QUERY = """
SELECT OFICIO,
				SUM(CASE WHEN DEPT_NO = 10 THEN SALARIO ELSE 0 END) AS DEPT_10,
				SUM(CASE WHEN DEPT_NO = 20 THEN SALARIO ELSE 0 END) AS DEPT_20,
				SUM(CASE WHEN DEPT_NO = 30 THEN SALARIO ELSE 0 END) AS DEPT_30,
				SUM(SALARIO) AS TOTAL
FROM EMPLEADO
GROUP BY OFICIO;
"""
df = pd.read_sql_query(QUERY, engine)
df

,OFICIO,DEPT_10,DEPT_20,DEPT_30,TOTAL
0,Analista,0,780000,0,780000
1,Director,318500,386750,370500,1075750
2,Empleado,169000,247000,123500,539500
3,Presidente,650000,0,0,650000
4,Vendedor,0,0,728000,728000


##### 29. Se desea saber para cada departamento y oficio, la suma total de comisiones, teniendo en cuenta que para los empleados que no tienen comisión, se les asignara:

- El 10% de su salario si son del departamento 10.
- El 15% de su salario si son del departamento 20.
- El 17% de su salario si son del departamento 30.
- Cualquier otro departamento, el 5% de su salario.


No se tendrá en cuenta a los empleados que hayan sido dados de alta después de 1981, ni al que ostente el cargo de "PRESIDENTE".

In [12]:
QUERY = """
SELECT DEPT_NO, OFICIO, SUM(CASE
							WHEN ISNULL(COMISION, 0) > 0 THEN COMISION
							WHEN ISNULL(COMISION,0) = 0 AND DEPT_NO = 10 THEN SALARIO*0.1
							WHEN ISNULL(COMISION,0) = 0 AND DEPT_NO = 20 THEN SALARIO*0.15
							WHEN ISNULL(COMISION,0) = 0 AND DEPT_NO = 30 THEN SALARIO*0.17
							ELSE SALARIO*0.05
							END) AS TOT_COM
FROM EMPLEADO
WHERE NOT (DATEPART(YEAR, FECHA_ALTA) > 1981 OR UPPER(OFICIO) IN ('PRESIDENTE'))
GROUP BY DEPT_NO, OFICIO
ORDER BY DEPT_NO;
"""
df = pd.read_sql_query(QUERY, engine)
df

,DEPT_NO,OFICIO,TOT_COM
0,10,Director,31850.0
1,20,Analista,58500.0
2,20,Director,58012.5
3,20,Empleado,15600.0
4,30,Director,62985.0
5,30,Empleado,20995.0
6,30,Vendedor,319150.0


##### 30.- Queremos saber el máximo, el mínimo y la media salarial, de cada departamento de la empresa

In [13]:
QUERY = """
SELECT DEPT_NO, MAX(SALARIO) AS MAXIMO, MIN(SALARIO) AS MINIMO, AVG(SALARIO) AS PROMEDIO
FROM EMPLEADO
GROUP BY DEPT_NO;
"""
df = pd.read_sql_query(QUERY, engine)
df

,DEPT_NO,MAXIMO,MINIMO,PROMEDIO
0,10,650000,169000,379166
1,20,390000,104000,282750
2,30,370500,123500,203666


In [14]:
# por buena practica, asegurarse de que la conexion se ciere
engine.dispose()

# Autor
### Oscar Gutierrez Leal
20 - 03 - 2024